In [2]:
import os

In [3]:
Start_Pages = ["Login_page.py","Welcome_page.py"]
Manager_Pages = ["Manager_main_page.py","Manager_data_page.py","Manager_account_manage_page.py"]
Owner_Pages = ["Owner_main_page.py","Owner_data_page.py","Subaccount_Management_Page.py","Owner_account_management_page.py",]
Common_Pages = ['Manager_account_signup_page.py','Verfication_page.py',"ProACT_Pro_page.py","Contact_us_page.py"]

folder_path = "UIs/Subpages"

tree = {"Start":Start_Pages,
            "Manager":Manager_Pages,
            "Owner":Owner_Pages,
            "Common":Common_Pages}


In [8]:

#create_page_files

for folder,pages in tree.items():
    for page in pages:
        folder_path_ = os.path.join(folder_path,page)
        #folder_path_+=".py"
        with open(folder_path_,"w") as f:
            f.write("import streamlit as st\n\n")
            content = f"""st.title("{page}")\n"""
            f.write(content)


In [15]:
# create UI file
with open("UI_.py","w") as f:
    for folder,pages in tree.items():
        for page in pages:
            line = f"from {folder_path}.{folder}.{page[:-3]} import {page[:-3]}"
            f.write(line+"\n")

In [ ]:
import pandas as pd

user_data = {"owner":{"password":'11',"role":"Owner",'Pro':"False",'email':'yzouaw@connect.ust.hk'},'manager1':{"password":'11',"role":"Manager",'Pro':"False",'email':'yzouaw@connect.ust.hk'},'manager2':{"password":'11',"role":"Manager",'Pro':"False",'email':'yzouaw@connect.ust.hk'}}
users = user_data.keys()
apis = {user : {"API":1} for user in users}
subaccount = {'owner' : ["manager1","manager2"]}
pd.DataFrame(user_data).to_csv('data/user_data.csv')
pd.DataFrame(apis).to_csv('data/api.csv')
pd.DataFrame(subaccount).to_csv('data/subaccount.csv')
pd.read_csv('data/user_data.csv',index_col=0)
pd.read_csv('data/api.csv',index_col=0)
pd.read_csv('data/subaccount.csv',index_col=0).to_dict()

{'owner': {0: 'manager1', 1: 'manager2'}}

In [4]:

import pandas as pd
from py2neo import Graph, Node, Relationship

user_data = {"owner":{"password":'11',"role":"Owner",'Pro':"False",'email':'yzouaw@connect.ust.hk'},'manager1':{"password":'11',"role":"Manager",'Pro':"False",'email':'yzouaw@connect.ust.hk'},'manager2':{"password":'11',"role":"Manager",'Pro':"False",'email':'yzouaw@connect.ust.hk'}}
users = user_data.keys()
apis = {user : {"API":1} for user in users}
subaccount = {'owner' : ["manager1","manager2"]}

uri = "bolt://localhost:7687"  # Neo4j 默认本地地址
user = "neo4j"                 # 默认用户名
password = '88888888'    # 替换为你设置的密码

db = Graph(uri,auth = (user,password))
for user,property in user_data.items():
        property['account'] = user
        node = Node(property['role'],**property)
        db.create(node)
        print(node)

(_49:Owner {Pro: 'False', account: 'owner', email: 'yzouaw@connect.ust.hk', password: '11', role: 'Owner'})
(_50:Manager {Pro: 'False', account: 'manager1', email: 'yzouaw@connect.ust.hk', password: '11', role: 'Manager'})
(_51:Manager {Pro: 'False', account: 'manager2', email: 'yzouaw@connect.ust.hk', password: '11', role: 'Manager'})


In [9]:
db.create(Node('API',**{"key":"MSBA",'number':0}))

In [54]:
owner = db.nodes.match('Owner',account = 'owner').first()
manager = db.nodes.match('Manager')
api = db.nodes.match('API').first()
owner.remove_label("Owner")  # 移除旧标签
owner.add_label("User")   # 添加新标签
db.push(owner)  # 保存更改
for i in manager:
    i.remove_label("Manager")  # 移除旧标签
    i.add_label("User")   # 添加新标签
    db.push(i)

In [53]:
api = db.nodes.match('API').first()
db.create(Relationship(api,'assign',owner))

In [62]:
for i in db.nodes.match('User'):
    print(i['account'])

owner
manager1
manager2


In [5]:
uri = "bolt://localhost:7687"  # Neo4j 默认本地地址
user = "neo4j"                 # 默认用户名
password = '88888888'    # 替换为你设置的密码

db = Graph(uri,auth = (user,password))
node = db.nodes.match('API').first()
node['number'] = 1
db.push(node)

db.create(Node('API', key = 'MSBA1',number=0))

db.create(Node('API', key = 'MSBA2',number=0))



In [94]:
node = db.nodes.match('User',account="owner3").first()
db.delete(node)

In [91]:
db.nodes.match("User", account='owner').first()

Node('Owner', 'User', Pro='False', account='owner', email='yzouaw@connect.ust.hk', password='11', role='Owner')

In [3]:

db.run("MATCH (n) DETACH DELETE n")

print("数据库中的所有数据已成功删除！")

数据库中的所有数据已成功删除！


In [4]:

import pandas as pd
from py2neo import Graph, Node, Relationship

uri = "bolt://localhost:7687"  # Neo4j 默认本地地址
user = "neo4j"                 # 默认用户名
password = '88888888'    # 替换为你设置的密码

db = Graph(uri,auth = (user,password))
api = Node("API", key = "MSBA",number = 0)
db.merge(api, "API", "key")
cmodel = Node("Model",type = "Central", modelpath = "Model/central_model_rf.pkl")
db.merge(cmodel, "Model", "type")

In [60]:
import pandas as pd
from py2neo import Graph

# 初始化 Neo4j 数据库连接
uri = "bolt://localhost:7687"  # Neo4j 默认本地地址
user = "neo4j"                 # 默认用户名
password = '88888888'          # 替换为你设置的密码

db = Graph(uri, auth=(user, password))

def restore_df_from_neo4j(node):
    # 获取所有餐厅节点
    data = [node]
    
    for restaurant in data:
        restaurant_id = restaurant["ID"]
        restaurant_type = restaurant["type"]
        restaurant_address = restaurant["Address"]

        # 获取与餐厅相关的销售记录节点（通过"sells"关系）
        dishes = db.match((restaurant,), r_type="sells").all()

        for dish in dishes:
            record_nodes = db.match((dish.end_node,), r_type="sold").all()
            
            for record_node in record_nodes:
                record_node = record_node.end_node
                record_time = record_node["time"]
                record_number = record_node["number"]
                record_discount = record_node["discount"]
                record_price = record_node["price"]
                #print(record_time,record_number,record_discount,record_price)

                # 获取相关的天气（Weather）
                weather = db.match((record_node,), r_type="with").first()
                weather_data = weather.end_node["weather"] 
                # 获取相关的事件（Event）
                event = db.match((record_node,), r_type="during").first()
                event_data = event.end_node["event"]

            # 获取相关的菜品（Dish）
            dish_name = dish["name"] if dish else None
            dish_id = dish["ID"] if dish else None
            
            
            
            # 将每一条记录添加到数据列表中
            data.append({
                "Restaurant ID": restaurant_id,
                "Restaurant Type": restaurant_type,
                "Restaurant Address": restaurant_address,
                "Name": dish_name,
                "ID": dish_id,
                "Date": record_time,
                "Number Sold": record_number,
                "Discount": record_discount,
                "Price": record_price,
                "Weather": weather_data,
                "Event": event_data
            })
    
    # 创建 DataFrame
    df = pd.DataFrame(data)
    return df

# 调用函数并返回 DataFrame
df = restore_df_from_neo4j('')
df

TypeError: string indices must be integers

In [71]:
import pandas as pd
from py2neo import Graph

# 初始化 Neo4j 数据库连接
uri = "bolt://localhost:7687"  # Neo4j 默认本地地址
user = "neo4j"                 # 默认用户名
password = '88888888'          # 替换为你设置的密码

db = Graph(uri, auth=(user, password))
def restore_df_from_neo4j(node):
    # 获取所有餐厅节点
    data = []
    
    for restaurant in [node]:
        print(restaurant)
        restaurant_id = restaurant["ID"]
        restaurant_type = restaurant["type"]
        restaurant_address = restaurant["Address"]

        # 获取与餐厅相关的销售记录节点（通过"sells"关系）
        dishes = db.match((restaurant,), r_type="sells").all()

        for dish in dishes:
            record_nodes = db.match((dish.end_node,), r_type="sold").all()
            
            for record_node in record_nodes:
                record_node = record_node.end_node
                record_time = record_node["time"]
                record_number = record_node["number"]
                record_discount = record_node["discount"]
                record_price = record_node["price"]
                #print(record_time,record_number,record_discount,record_price)

                # 获取相关的天气（Weather）
                weather = db.match((record_node,), r_type="with").first()
                weather_data = weather.end_node["weather"] 
                # 获取相关的事件（Event）
                event = db.match((record_node,), r_type="during").first()
                event_data = event.end_node["event"]

            # 获取相关的菜品（Dish）
            dish_name = dish.end_node["name"] 
            dish_id = dish.end_node["ID"] 
            
            
            
            # 将每一条记录添加到数据列表中
            data.append({
                "Restaurant ID": restaurant_id,
                "Restaurant Type": restaurant_type,
                "Restaurant Address": restaurant_address,
                "Name": dish_name,
                "ID": dish_id,
                "Date": record_time,
                "Number Sold": record_number,
                "Discount": record_discount,
                "Price": record_price,
                "Weather": weather_data,
                "Event": event_data
            })
    
    # 创建 DataFrame
    df = pd.DataFrame(data)
    return df

def get_restaurant_from_owner(node):
    subaccounts = db.match((node,),r_type='owns').all()
    dfs = []
    for subaccount in subaccounts:
        restaurant = db.match((subaccount.end_node,),r_type='is').first().end_node
        dfs.append(restore_df_from_neo4j(restaurant))
    return pd.concat(dfs)
        
node = db.nodes.match('User',account="Pro").first()
df = get_restaurant_from_owner(node)

(_5:Restaurant {Address: 'Wan Chai', ID: 'R137', type: 'Bakery'})
(_205:Restaurant {Address: 'Sham Shui Po', ID: 'R138', type: 'Bar'})


In [175]:
def preprecess_enhanced_data(df,reference_columns):
        """
        Preprocesses data and ensures alignment with reference_columns.
        """
        # Convert Date to datetime if not already
        df["Date"] = pd.to_datetime(df["Date"])
        y = df['Number Sold']
        # Extract temporal features
        df["Month"] = df["Date"].dt.month
        df["Day"] = df["Date"].dt.day
        df["Weekday"] = df["Date"].dt.weekday
        df["Is_Weekend"] = df["Weekday"].isin([5, 6]).astype(int)
        
        # One-hot encode categorical variables
        df = pd.get_dummies(df, columns=["Weather", "Event", "Restaurant Type"], drop_first=True)
        
        # Align columns with the central model's features
        missing_cols = [col for col in reference_columns if col not in df.columns]
        for col in missing_cols:
            df[col] = 0
        

        #print(df.columns,reference_columns)
        df = df[reference_columns]
        df["Number Sold"] = y
        
        return df

def make_predict(df):
    import joblib
    dfs =[]
    for id,data in df.groupby('Restaurant ID'):
        data.Date = pd.Series([str(i)[:10] for i in data.Date.to_list()])
        node = db.nodes.match("Restaurant",ID = id).first()
        subaccount = db.match((None,node),r_type='is').first().start_node
        model = db.match((subaccount,),r_type='model').first().end_node.get('modelpath')
        model = joblib.load(model)
        # 预处理数据，确保数据列名与训练时的一致
        #print(data.columns)
        clean_data = preprecess_enhanced_data(data, model.feature_names_in_)
        #print(clean_data.columns)
        #print(cols)
        # 只保留训练时的特征列，并且按照训练时的顺序排序
        clean_data = clean_data[model.feature_names_in_]  # 确保只保留训练时的特征列，并且顺序一致
        #print("Columns in clean_data:", clean_data.columns)
        #print("Columns in cols:", cols)
        # 进行预测
        predictions = model.predict(clean_data)
        data['pred'] = predictions
        dfs.append(data)
    return pd.concat(dfs,ignore_index=True)  # 返回处理后的 DataFrame，或根据需要返回其他内容
        
make_predict(df)

/Users/ysomad/Library/Python/3.9/lib/python/site-packages/sklearn/base.py:376: InconsistentVersionWarning: Trying to unpickle estimator DecisionTreeRegressor from version 1.4.2 when using version 1.5.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/Users/ysomad/Library/Python/3.9/lib/python/site-packages/sklearn/base.py:376: InconsistentVersionWarning: Trying to unpickle estimator RandomForestRegressor from version 1.4.2 when using version 1.5.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/Users/ysomad/Library/Python/3.9/lib/python/site-packages/sklearn/base.py:376: InconsistentVersionWarning: Trying to unpickle estimator DecisionTreeRegressor from v

,Restaurant ID,Restaurant Type,Restaurant Address,Name,ID,Date,Number Sold,Discount,Price,Weather,Event,Month,Day,Weekday,Is_Weekend,pred
0,R137,Bakery,Wan Chai,Dish 4,P008,2024-06-01,1205,0.25,16.0,Sunny,nan,6,1,5,1,2266.790000
1,R137,Bakery,Wan Chai,Dish 4,P009,2024-07-01,619,0.20,43.0,Sunny,nan,7,1,0,0,1513.211423
2,R137,Bakery,Wan Chai,Dish 4,P010,2024-08-01,1044,0.25,26.0,Sunny,nan,8,1,3,0,1730.590063
3,R137,Bakery,Wan Chai,Dish 4,P011,2024-09-01,832,0.25,40.0,Sunny,nan,9,1,6,1,1656.149143
4,R137,Bakery,Wan Chai,Dish 4,P012,2024-10-01,778,0.05,32.0,Sunny,Golden Week,10,1,1,0,992.409598
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65,R138,Bar,Sham Shui Po,Dish 4,P038,2024-10-27,742,0.05,28.0,Sunny,nan,10,27,6,1,766.247127
66,R138,Bar,Sham Shui Po,Dish 4,P039,2024-10-28,613,0.00,18.0,Sunny,nan,10,28,0,0,634.038401
67,R138,Bar,Sham Shui Po,Dish 4,P040,2024-10-29,937,0.25,18.0,Sunny,nan,10,29,1,0,857.713809
68,R138,Bar,Sham Shui Po,Dish 4,P041,2024-10-30,658,0.00,12.0,Rainy,nan,10,30,2,0,625.164700


In [109]:
#pd.Series([str(i)[:10] for i in df.Date.to_list()])
dd = pd.read_excel("test_data/Central.xlsx")

In [2]:
import pandas as pd
from py2neo import Graph, Node, Relationship

In [12]:
uri = "bolt://localhost:7687"  # Neo4j 默认本地地址
user = "neo4j"                 # 默认用户名
password = '88888888'          # 替换为你设置的密码

uri = st.secrets['database']['uri'] # Neo4j 默认本地地址
user = st.secrets['database']['user']            # 默认用户名
password = st.secrets['database']['password']    # 替换为你设置的密码

db = Graph(uri,auth = (user,password))
api = Node("API", key = "MSBA",number = 0)
db.merge(api, "API", "key")

cmodel = Node("Model",type = "Central", modelpath = "Model/central_model_rf.pkl")
db.merge(cmodel, "Model", "type")

In [2]:
from py2neo import Graph, Node
uri = "bolt://localhost:7687"  # Neo4j 默认本地地址
user = "neo4j"                 # 默认用户名
password = '88888888'  

db = Graph(uri,auth = (user,password))
api = Node("API", key = "MSBA",number = 0)
db.merge(api, "API", "key")
        # 替换为你设置的密码


ConnectionUnavailable: Cannot open connection to ConnectionProfile('bolt://localhost:7687')

In [9]:
import streamlit as st
st.secrets["api"]["api_key"]

'd4954786fb0f4a3e8a85fe828fc079bf'

2024-12-19 06:54:11.299 Thread 'ThreadPoolExecutor-0_1': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-12-19 06:54:11.301 Thread 'ThreadPoolExecutor-0_1': missing ScriptRunContext! This warning can be ignored when running in bare mode.


In [13]:
D1 = pd.DataFrame([1,2,3],[1,2,3])
D1

,0
1,1
2,2
3,3


In [15]:
a = [1,2,3,4]  
a.append([7,8])  
a

[1, 2, 3, 4, [7, 8]]